## Load data

In [1]:
import pickle
import data_helper

Using TensorFlow backend.


In [2]:
data_mixed_word, labels = data_helper.load_mixed_testset("word")


sampling 1297 none and 602 abusive label from davidson
(1297, 40)
(602, 40)
split:test, label:none, data shape:(2594, 40)
split:test, label:abusive, data shape:(3798, 40)


In [3]:
data_mixed_char, labels = data_helper.load_mixed_testset("char")


sampling 1297 none and 602 abusive label from davidson
(1297, 140, 70)
(602, 140, 70)
split:test, label:none, data shape:(2594, 140, 70)
split:test, label:abusive, data shape:(3798, 140, 70)


abusive binary with davidson

In [4]:
data_word_d, labels = data_helper.load_abusive_binary("word", True)
data_char_d, labels = data_helper.load_abusive_binary("char", True)

added relabel to training set
split:train, label:none, data shape:(14236, 40)
split:train, label:abusive, data shape:(35590, 40)
split:test, label:none, data shape:(1714, 40)
split:test, label:abusive, data shape:(4378, 40)
split:valid, label:none, data shape:(1713, 40)
split:valid, label:abusive, data shape:(4375, 40)
added relabel to training set
split:train, label:none, data shape:(14236, 140, 70)
split:train, label:abusive, data shape:(35590, 140, 70)
split:test, label:none, data shape:(1714, 140, 70)
split:test, label:abusive, data shape:(4378, 140, 70)
split:valid, label:none, data shape:(1713, 140, 70)
split:valid, label:abusive, data shape:(4375, 140, 70)


abusive binary without davidson

In [5]:
data_word_w, _ = data_helper.load_abusive_binary("word", False)
data_char_w, _ = data_helper.load_abusive_binary("char", False)

added relabel to training set
split:train, label:none, data shape:(10864, 40)
split:train, label:abusive, data shape:(15665, 40)
split:test, label:none, data shape:(1297, 40)
split:test, label:abusive, data shape:(1899, 40)
split:valid, label:none, data shape:(1297, 40)
split:valid, label:abusive, data shape:(1897, 40)
added relabel to training set
split:train, label:none, data shape:(10864, 140, 70)
split:train, label:abusive, data shape:(15665, 140, 70)
split:test, label:none, data shape:(1297, 140, 70)
split:test, label:abusive, data shape:(1899, 140, 70)
split:valid, label:none, data shape:(1297, 140, 70)
split:valid, label:abusive, data shape:(1897, 140, 70)


## Evaluate Abusive Classifier(first-step)

In [6]:
from keras import backend as K
from keras.models import load_model
from sklearn.metrics import classification_report
import numpy as np

In [7]:
x_word = data_mixed_word["x_test"]
y_word = data_mixed_word["y_test"]

In [8]:
x_char = data_mixed_char["x_test"]
y_char = data_mixed_char["y_test"]

### Waasem Word

In [9]:
model = load_model("./logs/waasem/word/weights.03.hdf5")
preds = model.predict(x_word, batch_size=128)
print(classification_report(np.argmax(y_word, axis=1), np.argmax(preds, axis=1), digits=3, target_names=labels))
K.clear_session()

             precision    recall  f1-score   support

       none      0.692     0.570     0.625      2594
    abusive      0.328     0.453     0.381      1204

avg / total      0.577     0.533     0.548      3798



### Waasem Hybrid

In [10]:
model = load_model("./logs/waasem/hybrid/weights.04.hdf5")
preds = model.predict([x_char, x_word], batch_size=128)
print(classification_report(np.argmax(y_word, axis=1), np.argmax(preds, axis=1), digits=3, target_names=labels))
K.clear_session()

             precision    recall  f1-score   support

       none      0.695     0.564     0.623      2594
    abusive      0.332     0.468     0.388      1204

avg / total      0.580     0.533     0.548      3798



### Davidson Word

In [11]:
model = load_model("./logs/davidson/word/weights.02.hdf5")
preds = model.predict(x_word, batch_size=128)
print(classification_report(np.argmax(y_word, axis=1), np.argmax(preds, axis=1), digits=3, target_names=labels))
K.clear_session()

             precision    recall  f1-score   support

       none      0.909     0.907     0.908      2594
    abusive      0.801     0.805     0.803      1204

avg / total      0.875     0.875     0.875      3798



### Davidson Hybrid

In [12]:
model = load_model("./logs/davidson/hybrid/weights.02.hdf5")
preds = model.predict([x_char, x_word], batch_size=128)
print(classification_report(np.argmax(y_word, axis=1), np.argmax(preds, axis=1), digits=3, target_names=labels))
K.clear_session()

             precision    recall  f1-score   support

       none      0.919     0.893     0.906      2594
    abusive      0.782     0.830     0.805      1204

avg / total      0.875     0.873     0.874      3798



### Pretrain Word

In [15]:
model = load_model("/home/homes/jhpark/hate-speech/logs/pretrain/word/weights.05.hdf5")
preds = model.predict(x_word, batch_size=128)
print(classification_report(np.argmax(y_word, axis=1), np.argmax(preds, axis=1), digits=3, target_names=labels))
K.clear_session()

             precision    recall  f1-score   support

       none      0.917     0.882     0.899      2594
    abusive      0.766     0.828     0.796      1204

avg / total      0.869     0.865     0.867      3798



### Pretrain Hybrid

In [14]:
model = load_model("/home/homes/jhpark/hate-speech/logs/pretrain/hybrid/weights.12.hdf5")
preds = model.predict([x_char, x_word], batch_size=128)
print(classification_report(np.argmax(y_word, axis=1), np.argmax(preds, axis=1), digits=3, target_names=labels))
K.clear_session()

             precision    recall  f1-score   support

       none      0.918     0.925     0.921      2594
    abusive      0.836     0.821     0.829      1204

avg / total      0.892     0.892     0.892      3798



## Train Sexism-racism classifier(second-step)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline

from model.helper import calculate_metrics
import numpy as np
from data.preprocess import load_from_file

In [17]:
original_data = load_from_file("waasem", ["sexism", "racism"])

loaded preprocessed tweets for sexism:3152
loaded preprocessed tweets for racism:1649
loaded preprocessed tweets for sexism:394
loaded preprocessed tweets for racism:206
loaded preprocessed tweets for sexism:395
loaded preprocessed tweets for racism:207


In [18]:
data = {}
for split in original_data.keys():
    x = "x_" + split
    y = "y_" + split
    data[x] = None
    data[y] = []
    for i, label in enumerate(["sexism", "racism"]):
        _data = original_data[split][label]
        if data[x] is not None:
            data[x] += _data
        else:
            data[x] = _data
        print("split:%s, label:%s" % (split, label))
        data[y] += [i+1 for _ in range(len(_data))]

split:train, label:sexism
split:train, label:racism
split:test, label:sexism
split:test, label:racism
split:valid, label:sexism
split:valid, label:racism


In [19]:
data["x_train"] = [" ".join(str(v) for v in row) for row in data["x_train"]]
data["x_train"][0]

'<user> bet i can prove that you , like most feminists , oppose equality quickly , a b or c ? women against feminism <url>'

In [20]:
data["x_test"] = [" ".join(str(v) for v in row) for row in data["x_test"]]
data["x_test"][0]

'rt <user> : <user> oh no ! heaven forbid a man finds a woman visually appealing ! the horror ! grow up . not sexist free speech science'

In [21]:
data["x_valid"] = [" ".join(str(v) for v in row) for row in data["x_valid"]]
data["x_valid"][0]

"<user> being a fuckface to a girl who just agreed , she doesn't want a feminazi hoe in her business , so fuck off ."

In [22]:
len(data["x_test"]), len(data["y_test"])

(602, 602)

In [23]:
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(2,5), analyzer="char")),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(solver="sag"))])    


In [24]:
text_clf.fit(data["x_train"], data["y_train"])

Pipeline(steps=[('vect', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 5), preprocessor=None, stop_words=None,
        strip... penalty='l2', random_state=None, solver='sag', tol=0.0001,
          verbose=0, warm_start=False))])

In [25]:
train_preds = text_clf.predict(data["x_train"])

In [26]:
print(classification_report(data["y_train"], train_preds, 
                            digits=3, target_names=["sexism", "racism"]))


             precision    recall  f1-score   support

     sexism      0.963     0.997     0.980      3152
     racism      0.993     0.927     0.959      1649

avg / total      0.973     0.973     0.972      4801



In [27]:
test_preds = text_clf.predict(data["x_test"])
print(classification_report(data["y_test"], test_preds, 
                            digits=3, target_names=["sexism", "racism"]))


             precision    recall  f1-score   support

     sexism      0.933     0.985     0.958       395
     racism      0.968     0.865     0.913       207

avg / total      0.945     0.944     0.943       602



## Evaluate Two-step classification

In [28]:
from data_helper import load_waasem, load_multiclass
from keras.utils import to_categorical

In [29]:
data_word_d = load_waasem("./data/word_outputs/")
data_char_d = load_waasem("./data/char_outputs/")

In [30]:
raw_data = load_from_file("waasem", ["none", "sexism", "racism"])

loaded preprocessed tweets for none:10376
loaded preprocessed tweets for sexism:3152
loaded preprocessed tweets for racism:1649
loaded preprocessed tweets for none:1297
loaded preprocessed tweets for sexism:394
loaded preprocessed tweets for racism:206
loaded preprocessed tweets for none:1297
loaded preprocessed tweets for sexism:395
loaded preprocessed tweets for racism:207


In [31]:
with open("./data/word_outputs/vocab.pkl", "rb") as f:
    vocab = pickle.load(f)

In [32]:
[" ".join([vocab["id2word"][token] for token in row]) for row in data_word_d["test"]["sexism"][-5:]]

['. <user> <user> <user> . <repeat> feminism would only have a name issue . PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD',
 '<user> please answer . <repeat> <url> PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD',
 'suck shit kat you vile , UNK , nasty bitch ! giving shit to annie and lloyd , only to be told your dish was gross ! karma bitch ! mkr PAD PAD PAD PAD PAD PAD PAD PAD PAD',
 "rt <user> : and don't call me sexist for that last tweet . if women want equal praise for success , they gotta take equal blame for UNK … PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD PAD",
 "rt <user> : some woman was proper staring at me whilst i was parking my car . it's ok babe , i male therefore i can drive . not sexist truth PAD PAD PAD PAD PAD PAD PAD PAD PAD"]

In [33]:
[" ".join(row) for row in raw_data["test"]["sexism"][-5:]]

['. <user> <user> <user> . <repeat> feminism would only have a name issue .',
 '<user> please answer . <repeat> <url>',
 'suck shit kat you vile , venomous , nasty bitch ! giving shit to annie and lloyd , only to be told your dish was gross ! karma bitch ! mkr',
 "rt <user> : and don't call me sexist for that last tweet . if women want equal praise for success , they gotta take equal blame for failur …",
 "rt <user> : some woman was proper staring at me whilst i was parking my car . it's ok babe , i male therefore i can drive . not sexist truth"]

In [34]:
x_word = None
x_char = None
x_text = []
y_test = []
for i, label in enumerate(["none", "sexism", "racism"]):
    assert len(data_word_d["test"][label]) == len(data_char_d["test"][label]) 
    assert len(data_word_d["test"][label]) == len(raw_data["test"][label])
    if x_word is None:
        x_word = data_word_d["test"][label]
    else:
        x_word = np.vstack((x_word, data_word_d["test"][label]))
    print(x_word.shape)
    
    if x_char is None:
        x_char = data_char_d["test"][label]
    else:
        x_char = np.vstack((x_char, data_char_d["test"][label]))
    print(x_char.shape)
    
    x_text += raw_data["test"][label]
    
    y_test += [i for _ in range(len(raw_data["test"][label]))]
y_test = to_categorical(y_test)

(1297, 40)
(1297, 140, 70)
(1692, 40)
(1692, 140, 70)
(1899, 40)
(1899, 140, 70)


### Waasem Word

In [35]:
model = load_model("./logs/waasem/word/weights.03.hdf5")
preds = np.argmax(model.predict(x_word, batch_size=128), axis=1)
for i, pred in enumerate(preds):
    if pred != 0:
        preds[i] = text_clf.predict([" ".join(x_text[i])])
print(classification_report(np.argmax(y_test, axis=1), preds, 
                            digits=3, target_names=["none", "sexism", "racism"]))
K.clear_session()

             precision    recall  f1-score   support

       none      0.688     0.528     0.597      1297
     sexism      0.254     0.476     0.331       395
     racism      0.534     0.420     0.470       207

avg / total      0.581     0.506     0.528      1899



### Waasem Hybrid

In [36]:
model = load_model("./logs/waasem/hybrid/weights.04.hdf5")
preds = np.argmax(model.predict([x_char, x_word], batch_size=128), axis=1)
for i, pred in enumerate(preds):
    if pred != 0:
        preds[i] = text_clf.predict([" ".join(x_text[i])])
print(classification_report(np.argmax(y_test, axis=1), preds, 
                            digits=3, target_names=["none", "sexism", "racism"]))
K.clear_session()

             precision    recall  f1-score   support

       none      0.700     0.527     0.602      1297
     sexism      0.260     0.491     0.340       395
     racism      0.542     0.464     0.500       207

avg / total      0.591     0.513     0.536      1899



### Davidson Word

In [37]:
model = load_model("./logs/davidson/word/weights.01.hdf5")
preds = np.argmax(model.predict(x_word, batch_size=128), axis=1)
for i, pred in enumerate(preds):
    if pred != 0:
        preds[i] = text_clf.predict([" ".join(x_text[i])])
print(classification_report(np.argmax(y_test, axis=1), preds, 
                            digits=3, target_names=["none", "sexism", "racism"]))
K.clear_session()

             precision    recall  f1-score   support

       none      0.838     0.899     0.867      1297
     sexism      0.720     0.572     0.638       395
     racism      0.705     0.657     0.680       207

avg / total      0.799     0.805     0.799      1899



### Davidson Hybrid

In [38]:
model = load_model("./logs/davidson/hybrid/weights.02.hdf5")
preds = np.argmax(model.predict([x_char, x_word], batch_size=128), axis=1)
for i, pred in enumerate(preds):
    if pred != 0:
        preds[i] = text_clf.predict([" ".join(x_text[i])])
print(classification_report(np.argmax(y_test, axis=1), preds, 
                            digits=3, target_names=["none", "sexism", "racism"]))
K.clear_session()

             precision    recall  f1-score   support

       none      0.865     0.876     0.870      1297
     sexism      0.691     0.656     0.673       395
     racism      0.697     0.710     0.703       207

avg / total      0.811     0.812     0.811      1899



### pretrain word

In [39]:
model = load_model("/home/homes/jhpark/hate-speech/logs/pretrain/word/weights.05.hdf5")
preds = np.argmax(model.predict(x_word, batch_size=128), axis=1)
for i, pred in enumerate(preds):
    if pred != 0:
        preds[i] = text_clf.predict([" ".join(x_text[i])])
print(classification_report(np.argmax(y_test, axis=1), preds, 
                            digits=3, target_names=["none", "sexism", "racism"]))
K.clear_session()

             precision    recall  f1-score   support

       none      0.867     0.873     0.870      1297
     sexism      0.679     0.663     0.671       395
     racism      0.696     0.696     0.696       207

avg / total      0.809     0.810     0.809      1899



### pretrain hybrid

In [40]:
model = load_model("/home/homes/jhpark/hate-speech/logs/pretrain/hybrid/weights.12.hdf5")
preds = np.argmax(model.predict([x_char, x_word], batch_size=128), axis=1)
for i, pred in enumerate(preds):
    if pred != 0:
        preds[i] = text_clf.predict([" ".join(x_text[i])])
print(classification_report(np.argmax(y_test, axis=1), preds, 
                            digits=3, target_names=["none", "sexism", "racism"]))
K.clear_session()

             precision    recall  f1-score   support

       none      0.859     0.911     0.884      1297
     sexism      0.736     0.648     0.689       395
     racism      0.766     0.647     0.702       207

avg / total      0.823     0.828     0.824      1899

